In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from triplet_preparation import inputs_from_tuples, tuples_from_file_array, train_inputs_file_array_generator, train_inputs_length
from model_architecture import triplet_network_model

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

tf.__version__, tf.keras.__version__

In [ ]:
%%time
'''Test Dataset'''

files = [
    os.path.abspath('../data/samples/lichess_db_standard_rated_2020-02-06-tuples-strong.h5')
]
# get triplets
triplets = tuples_from_file_array(files, "tuples", tuple_indices=[0, 1, 2])
# prepare for nn
train_triplets, test_triplets = inputs_from_tuples(triplets, test_split=True, test_size=0.05)
len(train_triplets),train_triplets[0].shape, len(test_triplets), test_triplets[0].shape

In [ ]:
'''Initialize and use triplet network'''
input_shape = (773,)
embedding_size = 10
model = triplet_network_model(input_shape, embedding_size)
optimizer = keras.optimizers.Adam(lr = 0.00006)
model.compile(loss=None,optimizer=optimizer)
model.summary()

In [11]:
files = [
    os.path.abspath('../data/samples/lichess_db_standard_rated_2020-02-06-tuples-strong.h5')
]
train_len = train_inputs_length(files, table_id_prefix="tuples")
print(f"{train_len} training samples.")

#train_generator = train_inputs_file_array_generator(files, table_id_prefix="tuples",
#					tuple_indices=[0,1,2], batch_size=64)

#history = model.fit(train_generator,
#                    steps_per_epoch=1000,
#                    epochs=int(train_len/1000/64)



324785 training sample.


In [ ]:
history = model.fit(train_triplets,
                    batch_size=64,
                    epochs=3) #,
                    #validation_data=([anc_test, pos_test, neg_test]))

print('\nhistory dict:', history.history)


In [ ]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate([anc_test, pos_test, neg_test], batch_size=128)
print('test loss:', results)

### OLD

In [ ]:
'''Create Checkpoints during training'''

checkpoint_path = "checkpoints/test_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
history = model.fit(train_images, 
                    train_labels,  
                    epochs=10,
                    validation_data=(test_images,test_labels),
                    callbacks=[cp_callback])  # Pass callback to training


In [ ]:
'''Loads the weights from checkpoint path'''
model = create_model()
model.load_weights(checkpoint_path)

In [ ]:
'''Save the complete model after training'''
model.save('model/test_model', save_format='tf') # or save_format='h5'

In [ ]:
'''Load model from saved state'''
model = tf.keras.models.load_model('model/test_model')
model.summary()

In [ ]:
'''Use loaded model for inference'''
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))
print(model.predict(test_images).shape)